# An example of reading from Kafka using Python consumer

In [2]:
# a copy of cluster parameters
# use Confluent webpage to get that info
params = {
    # take this from "Cluster Overview->Cluster Settings->General->Identification"
    "bootstrap_servers": "pkc-4r297.europe-west1.gcp.confluent.cloud:9092",
    "security_protocol": "SASL_SSL",
    "sasl_mechanism": "PLAIN",
    # Data Integration->API Keys->Add key
    # username = key
    "sasl_plain_username": "4KLAOHXKUM6GFMLJ",
    # password = secret
    "sasl_plain_password": "tdpFCiSB+pMISzXXsLnkBeYZAKc9+lb4rIRWcuv7simAlUWUbqRi8KxUt21w5+XY"
}

In [3]:
MAIN_TOPIC = "input_topic"

In [4]:
from kafka import KafkaConsumer

consumer = KafkaConsumer(**params)

In [5]:
from kafka import TopicPartition

partitions = [
    TopicPartition(MAIN_TOPIC, partition)
    for partition in consumer.partitions_for_topic(MAIN_TOPIC)
]

In [6]:
from kafka import KafkaProducer

producer = KafkaProducer(**params)

In [7]:
consumer.assign(partitions)
consumer.seek(partitions[0], 0)
while True:
    try:
        data = consumer.poll(
            timeout_ms=200,
            # take at most 1024 events
            max_records=1024
        )[partitions[0]]
        result = sum([float(item.value.decode("utf-8")) for item in data]) / 1024
        producer.send(
            "output_topic",
            str(result).encode("utf-8")
        )
    except KeyError as error:
        # this helps to ignore delays in the input stream
        pass

KeyboardInterrupt: 